In [3]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error 
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [4]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

def compute_spread(p1, p2, tf, x):
    data1 = get_rates(p1, tf, x)
    data2 = get_rates(p2, tf, x)
    merged = data1.join(data2, lsuffix="_x", rsuffix="_y")
    spread = merged['close_x'] - merged['close_y']
    return spread.dropna()

def adf_test(spread):
    '''Runs ADF test on a spread series'''
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'granger_xgboost',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)

def send_order(symbol, side, lot, comment):

    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    result

def check_close_position(trade_position):
    from datetime import datetime
    from datetime import datetime, timedelta
    # Your logic to close the position
    ticket = trade_position.ticket
    time = datetime.fromtimestamp(trade_position.time)
    print(time)
    symbol = trade_position.symbol
    comment = trade_position.comment
    # print(f"Checking {symbol} of {comment} to close")
    # Sample trade position (replace this with the actual object you get)
    position = {
        'ticket': ticket,
        'time': time,  # This would be the actual Unix timestamp
        'Symbol': symbol,
        'Comment': comment,
    }

    local_to_utc_offset = timedelta(hours= +3)

    # Get the current time in GMT+3
    current_time_gmt_plus_3 = datetime.now() 

    # Convert it to UTC
    current_time_utc = current_time_gmt_plus_3 + local_to_utc_offset

    # Convert to Unix timestamp
    current_unix_time = int(current_time_utc.timestamp())

    # Calculate the time difference in seconds
    time_difference = current_unix_time - int(i.time)
    
    # Check if 4 hours or more have passed (4 hours = 4 * 60 * 60 seconds)
    if time_difference >= 86400:
        print(f"Closing {symbol} of {comment}")
        close_position(trade_position)
    else:
        print(f"Position {symbol} of {comment} has been open for less than 24 hours. Time open: {round(round((time_difference / 60),2) / 60), 4} hrs")

In [14]:
from datetime import datetime, timedelta

def check_close_position(trade_position):
    # Assuming trade_position.time is a Unix timestamp in UTC
    time = datetime.fromtimestamp(trade_position.time)
    
    print(f"Trade opened at: {time}")
    
    current_utc_time = datetime.utcnow()
    
    # Convert to GMT+2
    current_gmt2_time = current_utc_time + timedelta(hours=2)
    
    # Calculate the time difference in seconds
    time_difference = (current_gmt2_time - time).total_seconds()
    
    # Check if 24 hours or more have passed (24 hours = 24 * 60 * 60 seconds)
    if time_difference >= 86400:
        print(f"--------------------- Closing {trade_position.symbol} of {trade_position.comment}")
        # close_position(trade_position)
    else:
        hours_open = time_difference / 3600
        print(f"Position {trade_position.symbol} of {trade_position.comment} has been open for less than 24 hours. Time open: {round(hours_open, 2)} hrs")

# Sample Usage
# check_close_position(sample_trade_position)

In [17]:
for i in mt5.positions_get():
    if i.comment == 'grang_d':
        print(f"Checking {i.comment} of {i.symbol}")
        check_close_position(i)

Checking grang_d of GBPUSD.a
Trade opened at: 2023-11-15 23:01:39
--------------------- Closing GBPUSD.a of grang_d
Checking grang_d of EURUSD.a
Trade opened at: 2023-11-15 23:01:40
--------------------- Closing EURUSD.a of grang_d
Checking grang_d of AUDCAD.a
Trade opened at: 2023-11-15 23:01:41
--------------------- Closing AUDCAD.a of grang_d
Checking grang_d of USDCAD.a
Trade opened at: 2023-11-15 23:01:41
--------------------- Closing USDCAD.a of grang_d
Checking grang_d of USDCAD.a
Trade opened at: 2023-11-15 23:01:42
--------------------- Closing USDCAD.a of grang_d
Checking grang_d of NZDCAD.a
Trade opened at: 2023-11-15 23:01:43
--------------------- Closing NZDCAD.a of grang_d
Checking grang_d of GBPCHF.a
Trade opened at: 2023-11-15 23:01:43
--------------------- Closing GBPCHF.a of grang_d
Checking grang_d of EURCHF.a
Trade opened at: 2023-11-15 23:01:44
--------------------- Closing EURCHF.a of grang_d
Checking grang_d of EURNOK.a
Trade opened at: 2023-11-15 23:01:46
------